# Pseudonym Detection

In [1106]:
import h5py
import json
import math
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib import rc
import datetime
import scipy.signal as signal

import statistics as stats
import os
import scipy.io
import h5py


In [1107]:
'''
Protocol parameters!!!
'''
packet = 6000
samples = 600
pseudonym_len = 28
TX_length = int(packet*pseudonym_len)
repeat = 10
message = 'STOP' # This is our pseudonym message we transmit.

In [1108]:
def get_time_string(timestamp):
    '''
    Helper function to get data and time from timestamp
    INPUT: timestamp
    OUTPUT: data and time. Example: 01-04-2023, 19:50:27
    '''
    date_time = datetime.datetime.fromtimestamp(int(timestamp))
    return date_time.strftime("%m-%d-%Y, %H:%M:%S")

In [1109]:
def JsonLoad(folder, json_file):
    '''
    Load parameters from the saved json file
    INPUT
    ----
        folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
        json_file: the json file with all the specifications. Example: '/save_iq_w_tx_gold.json'
    OUTPUT
    ----
        samps_per_chip: samples per chip
        wotxrepeat: number of repeating IQ sample collection w/o transmission. Used as an input to 
        traverse_dataset() func
        rxrate: sampling rate at the receiver side
    '''
    #config_file = folder+'/'+json_file
    #config_file = ""+"/"+str(folder)+"/save_iq_w_tx_file.json"
    config_dict = json.load(open(json_file))[0]
    nsamps = config_dict['nsamps']
    rxrate = config_dict['rxrate']
    rxfreq = config_dict['rxfreq']
    wotxrepeat = config_dict['wotxrepeat']
    rxrepeat = config_dict['rxrepeat']
    txnodes = config_dict['txclients']
    rxnodes = config_dict['rxclients']

    return rxrepeat, rxrate, txnodes, rxnodes

In [1110]:
def traverse_dataset(meas_folder):
    '''
    Load data from hdf5 format measurement file
    INPUT
    ----
        meas_folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
    OUTPUT
    ----
        data: Collected IQ samples w/ transmission. It is indexed by the transmitter name
        noise: Collected IQ samples w/o transmission. It is indexed by the transmitter name
        txrxloc: transmitter and receiver names
    '''
    data = {}
    noise = {}
    txrxloc = {}

    dataset = h5py.File(meas_folder + '/measurements.hdf5', "r") #meas_folder
    #print("Dataset meta data:", list(dataset.attrs.items()))
    for cmd in dataset.keys():
        #print("Command:", cmd)
        if cmd == 'saveiq':
            cmd_time = list(dataset[cmd].keys())[0]
           # print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for rx_gain in dataset[cmd][cmd_time].keys():
               # print("   RX gain:", rx_gain)
                for rx in dataset[cmd][cmd_time][rx_gain].keys():
                    print("")
                    #print("     RX:", rx)
                    #print("       Measurement items:", list(dataset[cmd][cmd_time][rx_gain][rx].keys()))
        elif cmd == 'saveiq_w_tx':
            cmd_time = list(dataset[cmd].keys())[0]
            #print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for tx in dataset[cmd][cmd_time].keys():
                #print("   TX:", tx)
                
                if tx == 'wo_tx':
                    for rx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("       RX gain:", rx_gain)
                       # print(dataset[cmd][cmd_time][tx][rx_gain].keys())
                        for rx in dataset[cmd][cmd_time][tx][rx_gain].keys():
                            #print("         RX:", rx)
                            #print("           Measurement items:", list(dataset[cmd][cmd_time][tx][rx_gain][rx].keys()))
                            repeat = np.shape(dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'])[0]
                            #print("         repeat", repeat)

                            samplesNotx =  dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'][:repeat, :]
                            namelist = rx.split('-')
                            noise[namelist[1]] = samplesNotx
                else:
                    for tx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("     TX gain:", tx_gain)
                        for rx_gain in dataset[cmd][cmd_time][tx][tx_gain].keys():
                            #print("       RX gain:", rx_gain)
                            #print(dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys())
                            for rx in dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys():
                                repeat = np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'])[0]
                                #print("         RX:", rx, "; samples shape", np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples']))
                                #print("         Measurement items:", list(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx].keys()))
                                # print("         rxloc", (dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxloc'][0]))
                                # peak avg check
                                
                                txrxloc.setdefault(tx, []).extend([rx]*repeat)
                                rxsamples = dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'][:repeat, :]
                                data.setdefault(tx, []).append(np.array(rxsamples))

        else:                       
            print('Unsupported command: ', cmd)

    return data, noise, txrxloc

In [1111]:
# PURPOSE: perform preamble synchronization
#          Uses the (complex-valued) preamble signal. The cross-correlation 
#          of the preamble signal and the received signal (at the time
#          when the preamble is received) should have highest magnitude
#          at the index delay where the preamble approximately starts.  
# INPUT:   rx0: received signal (with a frequency offset)
#          preambleSignal: complex, known, transmitted preamble signal 
# OUTPUT:  lagIndex: the index of rx0 where the preamble signal has highest 
#              cross-correlation
#
def crossCorrelationMax(rx0, preambleSignal):

    # Cross correlate with the preamble to find it in the noisy signal
    lags      = signal.correlation_lags(len(rx0), len(preambleSignal), mode='valid')
    xcorr_out = signal.correlate(rx0, preambleSignal, mode='valid')
    xcorr_mag = np.abs(xcorr_out)
    # Don't let it sync to the end of the packet.
    packetLenSamples = TX_length
    maxIndex = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex = lags[maxIndex]

    print('Max crosscorrelation with preamble at lag ' + str(lagIndex))

    # Plot the selected signal.
    plt.figure()
    fig, subfigs = plt.subplots(2,1)
    subfigs[0].plot(np.real(rx0), label='Real RX Signal')
    subfigs[0].plot(np.imag(rx0), label='Imag RX Signal')
    scale_factor = np.mean(np.abs(rx0))/np.mean(np.abs(preambleSignal))
    subfigs[0].plot(range(lagIndex, lagIndex + len(preambleSignal)), scale_factor*np.real(preambleSignal), label='Preamble')
    subfigs[0].legend()
    subfigs[1].plot(lags, xcorr_mag, label='|X-Correlation|')
    plt.xlabel('Sample Index', fontsize=14)
    plt.tight_layout()
    plt.show()

    return lagIndex

In [1112]:
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [1113]:
def binvector2str(binvector):
    #binvector = binvector[0]
    length = len(binvector)
    eps = np.finfo('float').eps
    if abs(length/7 - round(length/7)) > eps:
        print('Length of bit stream must be a multiple of 7 to convert to a string.')
    # Each character requires 7 bits in standard ASCII
    num_characters = round(length/7)
    # Maximum value is first in the vector. Otherwise would use 0:1:length-1
    start = 6
    bin_values = []
    while start >= 0:
        bin_values.append(int(math.pow(2,start)))
        start = start - 1
    bin_values = np.array(bin_values)
    bin_values = np.transpose(bin_values)
    str_out = '' # Initialize character vector
    for i in range(num_characters):
        single_char = binvector[i*7:i*7+7]
        value = 0
        for counter in range(len(single_char)):
            value = value + (int(single_char[counter]) * int(bin_values[counter]))
        str_out += chr(int(value))
    return str_out

In [1115]:
## Hard Decision Algorithm
## Make p-bit decisions by comparing patterns on bit-0 and bit-1
## Trace changes in power with the p-bit and compare it with the known chip pattern.
## This algorithm improves pseudonym detection in the presence of non-Gaussion type interferences

def Matched_Filter_RF_Mitigation_Algorithm(x):
    matching_filter0 =np.array([-1,1,-1,1,-1,1,-1,1,-1,1])
    matching_filter1 =np.array([1,-1,1,-1,1,-1,1,-1,1,-1])
    p_bit = []
    for i in range(28):
        pbit_data = x[i*packet:(i+1)*packet] # slices samples into one p-bit data = 6000 samples
        
        power = []
        Cr = []       
        threshold = 0.0
        quantization_level = np.array([1,-1])
        for j in range(10):
            chip_data = pbit_data[j*samples:(j+1)*samples] # slice p-bit data into chip data = 600 samples
            power.append(1000*sum(abs(chip_data)**2)/len(chip_data))


        for k in range(1,10):

            if power[k] > power[k-1]:
                Cr.append(quantization_level[0])
            else:
                Cr.append(quantization_level[1]) 

        if np.dot(Cr,matching_filter1[1:]) >= np.dot(Cr,matching_filter0[1:]): # p-bit decision done by comparing p-bit powers with the threshold.
            p_bit.append(1)
        else:
            p_bit.append(0) 
            
    return np.array(p_bit)

In [1116]:
## Soft Decision Algorithm
## Make p-bit decisions by comparing patterns on bit-0 and bit-1
## Trace changes in power with the p-bit and compare it with the known chip pattern.
## This algorithm improves pseudonym detection in the presence of non-Gaussion type interferences

def Matched_Filter_Pseudonym_Detection_Algorithm(x):
    matching_filter0 =np.array([-1,1,-1,1,-1,1,-1,1,-1,1])
    matching_filter1 =np.array([1,-1,1,-1,1,-1,1,-1,1,-1])
    p_bit = []
    for i in range(28):
        pbit_data = x[i*packet:(i+1)*packet] # slices samples into one p-bit data = 6000 samples
        
        power = []
        Cr = []       
        threshold = 0.0
        quantization_level = np.array([1,-1])
        for j in range(10):
            chip_data = pbit_data[j*samples:(j+1)*samples] # slice p-bit data into chip data = 600 samples
            power.append(1000*sum(abs(chip_data)**2)/len(chip_data))  
        
        if np.dot(power,matching_filter1) > np.dot(power,matching_filter0): # p-bit decision done by comparing p-bit powers with the threshold.
            p_bit.append(1)
        else:
            p_bit.append(0)   
    return np.array(p_bit)

In [1117]:
'''
calculates the distance between the recoded pseudonym bits and the transmitted(true) pseudonym bits.
'''
def Distance(X,Y):
    if len(X) != len(Y):
        print('Warning: arrays have different dimensions')
    count = 0
    for i in range(len(X)):
        if X[i]!= Y[i]:
            count +=1
    return count

In [1118]:
'''Use preamble to estimate start of OFDM packet'''
def Generate_HTSTF():
    data = scipy.io.loadmat('HTSTF.mat')
    new_data = data['stf'].reshape((1,len(data['stf'])))
    preamble = np.tile(new_data,10)[0]
    return preamble

In [1119]:
preambleSignal = Generate_HTSTF()

# Pseudonym Detection at Passive Receiver Below

In [1121]:
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [1122]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Calculate_Eb_No_Passive(x):
    
    # Load parameters from the JSON file which describe what was measured
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    # samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    # rxloc = 'cbrssdr1-bes-comp'
    rxloc = 'cbrssdr1-browning-comp'
    #Calculate SNR
    # Noise = rx_noise['bes'][0] 
    Noise = rx_noise['browning'][0]# measure the RX power while the TX is turned off
    noise_power = np.mean(abs(Noise)**2)
    
    P_s = 0 #received power

    for i in range(repeat):    
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][i]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]
       
        P_s += np.mean(abs(Rx_signal)**2) 
    return P_s/repeat, noise_power

In [1123]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''

def Calculate_Pseudonym_BER(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
#     samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    # rxloc = 'cbrssdr1-bes-comp'
    rxloc = 'cbrssdr1-browning-comp'
    # initialize error
    pseudonym_BER = 0
    P_s = 0
    count = 0
    for i in range(repeat):    
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][i]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]

        estimate_pseudonym = Matched_Filter_RF_Mitigation_Algorithm2(Rx_signal)
        print(binvector2str(estimate_pseudonym))

        pseudonym_BER += Distance(text2bits('STOP'),estimate_pseudonym)
    return pseudonym_BER

In [1125]:
'''
Computes the average pseudonym error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Probability_Pseudonym_Detection(x):
    folders = Extract_Folders(x)[0]
    num_folder = len(folders)
    pseudonym_BER = 0
    signal = 0
    noise = 0
    noise_power = []
    signal_power = []
    for i in range(len(folders)):
        s,n = Calculate_Eb_No_Passive(x + '/'+folders[i])
        noise_power.append(n)
        signal_power.append(s)
        signal += s
        noise += n
    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))

    # plt.plot(noise_power)
    # plt.show()
    # plt.plot(signal_power)
    # plt.show()
    print('The SNR at the Passive RX is:',round(Eb_No,2), 'dB \n')

    for i in range(len(folders)):
        print(folders[i])
        BER = Calculate_Pseudonym_BER(x + '/'+folders[i])
        pseudonym_BER += BER
    
    p_bit_error = pseudonym_BER/(num_folder*repeat*pseudonym_len)
    return p_bit_error, Eb_No

# Results Below

In [ ]:
Probability_Pseudonym_Detection("-13dB_03")